In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
from scipy import stats

*Después de leer los archivos CSV, se guardan en dos DataFrames: df_juegos y df_precios.*

In [ ]:
def convertir_a_lista(cadena):
    return eval(cadena)

df_juegos = pd.read_csv("juegos_ludonauta_airtable.csv", converters={'Autores': convertir_a_lista,
                                                                    'Categorias': convertir_a_lista,
                                                                    'Mecanicas' : convertir_a_lista,
                                                                    'Juegos relacionados' : convertir_a_lista})

df_precios = pd.read_csv('precios_ludonauta_airtable.csv', converters ={'Nombre tienda': convertir_a_lista,
                                                              'Precio': convertir_a_lista,
                                                              'Disponibilidad' : convertir_a_lista})

*Se añade la columna del nombre de los juegos al DataFrame de precios.*

In [ ]:
df_precio = pd.merge(df_juegos, df_precios, on='Link', how='inner')

columnas_borrar = ['Fecha publicacion', 'Autores', 'Categorias',
                'Mecanicas', 'Puntuacion', 'Jugadores', 'Jugadores minimos',
                'Jugadores maximos', 'Duracion', 'Duracion minima', 'Duracion maxima',
                'Edad', 'Complejidad', 'Dep idioma', 'Edicion juegos',
                'Juegos relacionados', 'Mediana duracion', 'Mediana jugadores'] 

df_precio = df_precio.drop(columnas_borrar, axis=1)


*Se muestra lo que contiene el df de precios.*

In [ ]:
df_precio = df_precio.drop(columns= {'Unnamed: 0_x', 'Unnamed: 0_y'}, axis=1)

df_precio.head()

*Se añade la columna de precio medio al DataFrame de juegos sin realizar un merge, ya que coinciden tanto el índice como el enlace con el DataFrame de precios.*

In [ ]:
df_juegos['Precio medio'] = df_precio['Precio medio']
df_juegos = df_juegos.drop('Unnamed: 0', axis=1)
df_juegos['Edicion juegos'] = df_juegos['Edicion juegos'].astype(str)
df_juegos['Edicion juegos'] = df_juegos['Edicion juegos'].apply(lambda x: eval(x)[0] if isinstance(eval(x), list) else x)

df_juegos.head(3)

*En describe se observa:*

- **Fecha de publicación:** *La media de creaciones de juegos es de 2016, relativamente reciente.*

- **Cantidad de jugadores:** *La mayoría están creados para un rango de  2 a 6 personas.*

- **Duración del juego:** *La mediana es de 45 minutos.* 

- **Edad recomendada:** *La mediana muestra que a partir de los 10-11 años están creados todos los juegos.*

- **Complejidad y Dep idioma:** *La media muestra que la gran mayoría es baja.*

- **Jugadores:** *Tiene una mediana de 3-4 jugadores, todos los juegos.*

- **Precio medio:** *La mediana muestra que los juegos tienen un precio medio de 26 euros y la media 33.*

In [ ]:
df_juegos.describe()

**Interpretación**
- Hay una tendencia reciente en la producción de juegos, sugiriendo un aumento en el interés en la industria de los juegos de mesa en los últimos años.
- La mayoría de los juegos están diseñados para 2 a 6 personas, destacando un enfoque en grupos más pequeños.
- Ofrecen una experiencia de juego de duración moderada, adaptable a un tiempo limitado disponible.
- Adecuados tanto para niños como para adultos.
- Presentan baja complejidad y dependencia del idioma, haciéndolos accesibles para un público amplio.
- El número de jugadores indica su idoneidad para grupos de tamaño estándar.
- El rango de precios está dentro de lo razonable para lo que los consumidores pueden esperar pagar por estos juegos.

En resumen, los juegos de mesa se centran en proporcionar una experiencia diversa y accesible, con énfasis en grupos más pequeños y duración moderada, lo que los hace atractivos para una amplia audiencia.

*Para iterar sobre las columnas que contienen listas, se emplea el método EXPLODE, el cual descompone las filas en tantas filas como elementos contengan las listas. Finalmente, se crea un nuevo DataFrame denominado df_explode.*

*Al agrupar los datos de ese DF, en ciertas partes del notebook se utiliza la función de agregación enyunique, para extraer los números de juegos únicos.*

In [ ]:
df_explode = df_juegos.explode("Categorias").explode("Autores").explode("Mecanicas").explode("Juegos relacionados").reset_index(drop = True)
df_explode.head(3)

## ANÁLISIS DE LOS AUTORES

*La autoría de un juego puede atribuirse a uno o más autores, habiendo un total de 3985 y 583 juegos sin autor asignado. Se crea un df del número de juegos que ha creado cada autor, mostrando los 20 autores que más han creado.*

In [ ]:
conteo_autores = {}

for lista_autores in df_juegos["Autores"]:
    if isinstance(lista_autores, list):
        for autor in lista_autores:
            if autor in conteo_autores:
                conteo_autores[autor] += 1
            else:
                conteo_autores[autor] = 1

df_autores_n_juegos = pd.DataFrame(list(conteo_autores.items()), columns=["Autores", "Numero juegos"])
df_autores_n_juegos = df_autores_n_juegos.sort_values(by="Numero juegos", ascending=False).head(21)
df_autores_n_juegos

*Se crea un gráfico de barras para representarlo (quitando Desconocido). Se puede observar la gran cantidad de juegos creados por Eric M. Lang.*

In [ ]:
fig= px.bar(data_frame = df_autores_n_juegos.iloc[1:],
       x          = "Autores",
       y          = "Numero juegos",
       opacity     = 0.5,
       title = "Top 20 autores",
       height = 700,
       color_discrete_sequence= ['red']
      )
fig.update_layout(title_x=0.5)

**Interpretación**
- Un significativo número de juegos (583) carece de autor identificado, sugiriendo una diversidad de orígenes o una falta de información sobre los creadores.
- Resalta la notable contribución de ciertos autores, siendo Eric M. Lang el que más destaca con el mayor número de juegos.

*Se observan algunos de los juegos que ha creado el autor con el mayor número de juegos.*

In [ ]:
df_explode[df_explode["Autores"] == "Eric M. Lang"].drop_duplicates("Link").head()

*Se crea un listado de los juegos creados por Eric M. Lang.*

In [ ]:
df_explode[df_explode["Autores"] == "Eric M. Lang"].drop_duplicates("Link")["Nombre"].to_list()

**Interpretación**

Se observa que Eric M. Lang ha creado una extensa lista de juegos de mesa que abarcan diferentes temáticas, sagas y universos, como Cthulhu, Marvel, Blood Rage, Juego de Tronos o Warhammer... 

Destaca en categorías como Cartas, Fantasía y Ciencia ficción, con mecanicas como Gestión de mano, Lanzamiento de dados, y Habilidades variables. Categorías y mecánicas que veremos más adelante que son tendencias notables y destacadas en la evolución actual de los juegos de mesa.

## NOMBRES DE LOS JUEGOS

*Se muestran las palabras más populares (>100) a la hora de nombrar juegos. 

"LCG" significa Living Card Game, se sabe exactamente qué cartas se están obteniendo en cada expansión. 

In [ ]:
palabras_a_excluir = ["de", "–", "of", "The", "El", "el", "La", "the", "y", "del", "La", "la", "&", "los", "Los", "en"]

conteo_palabras = df_juegos["Nombre"].str.split().explode().value_counts().sort_values(ascending=False)
conteo_palabras_filtrado = conteo_palabras[(~conteo_palabras.index.isin(palabras_a_excluir)) & (conteo_palabras > 100)]
df_palabras_populares= pd.DataFrame(conteo_palabras_filtrado)
df_palabras_populares

**Interpretación**

Las tendencias notables en juegos de mesa incluyen un interés en juegos de cartas coleccionables (LCG), franquicias populares como Star Wars y Marvel, y temáticas de fantasía, especialmente relacionadas con la serie Juego de Tronos.

*Por qué letras empiezan más los nombres de los juegos.*

In [ ]:
primeras_letras = df_juegos["Nombre"].str[0].str.upper()

conteo_letras = primeras_letras.value_counts().sort_values(ascending=False)
conteo_letras.head()

**Interpretación**

Al diseñar un juego de mesa, la creación de un nombre original implica considerar letras o palabras iniciales diferentes, lo cual contribuirá a resaltar y hacer más memorable la identidad del juego.

## EDICIÓN DE JUEGOS

- **Juego de Mesa Base:** *La edición principal del juego. Cuando en juegos relacionados pone "Ninguno", es porque es el juego principal.*

- **Expansión:** *Una edición que añade contenido adicional al juego base, como nuevas reglas, componentes o temáticas.*

- **Integración:** *Una edición que se integra con otras ediciones o juegos, permitiendo combinarlos para una experiencia más amplia.*

- **Reimplementación:** *Una edición que conserva algunas mecánicas clave pero presenta cambios significativos en el tema, diseño o reglas.*

*Recuento de juegos para cada tipo de edición.*

In [ ]:
n_edicion_juegos = df_juegos.groupby("Edicion juegos")["Juegos relacionados"].count().reset_index(name="Count")
n_edicion_juegos

*Representación de la edición de juegos mediante un gráfico de pastel.*

In [ ]:
fig= px.pie(data_frame = n_edicion_juegos,
       names      = "Edicion juegos",
       title      = "Porcentajes de edición de juegos",
       values     = "Count",
       opacity    = 1)
fig.update_layout(title_x=0.5)

**Interpretación**

La diversidad en las categorías de ediciones de juegos revela distintas estrategias y enfoques en la industria de los juegos de mesa, desde expansiones y juegos base hasta integraciones y reimplementaciones. 

La presencia considerable de juegos de mesa base sugiere que sigue siendo una parte fundamental en la industria de juegos de mesa. El porcentaje de expansión refleja una tendencia a desarrollar y lanzar contenido adicional para juegos existentes.

*Se crea un gráfico de sol para observar el porcentaje para los 50 juegos con mayor puntuación creados por cada autor para cada tipo de edición, teniendo en cuenta que pueden aparecer en varios tipos.*

In [ ]:
df = df_explode.groupby(['Edicion juegos','Autores'], as_index= False).agg({('Puntuacion') : 'max'}).sort_values('Puntuacion', ascending = False).head(50)


fig = px.sunburst( data_frame  = df,
                    values     = 'Puntuacion',
                    path       = ['Edicion juegos', 'Autores'],
                    color      = 'Puntuacion',
                    height= 750,
                    title= "Porcentaje de Tipos de Edición en los 50 Juegos Mejor Puntuados por Autor"
                )
fig.update_layout(margin=dict(l=0, r=0, b=8, t=80), title_x=0.5)
fig.update_traces(textinfo="label+percent parent", branchvalues="total")
fig.show()

**Interpretación**

Los autores que desarrollaron reimplementaciones y expansiones obtuvieron una puntuación más alta, posiblemente debido a que estos productos suelen ser adquiridos después del juego de mesa base, ya que los jugadores conocen y disfrutan del juego original, lo que contribuye a un aumento en la puntuación.

Los autores más destacados coinciden con el análisis de las puntuaciones por nombre de juego que aparece más adelante. Es notable la ausencia de Eric M. Lang entre los 50 con mejor puntuación; sugiriendo que la calidad supera a la cantidad.

Jacob Fryxelius sobresale con la mejor puntuación entre todos los demás. Es un autor sueco creador del Terraforming Mars, que es un juego de gestión de cartas y recursos en el que se juega para conseguir terraformar Marte. 

*Juegos creados por Jacob Fryxelius*

In [ ]:
df_explode[df_explode["Autores"] == "Jacob Fryxelius"].drop_duplicates("Link")["Nombre"].to_list()

## ANÁLISIS DE LOS JUEGOS RELACIONADOS

* Se crea un DataFrame del número de veces que aparece cada juegos relacionados y se muestran los 15 más populares.*
- *Hay un total de 1729 juegos relacionados y 5588 juegos base (pone "Ninguno" en juegos relacionados porque es el juego principal).*

In [ ]:
conteo_juegos_relacionados = {}

for lista_juegos_relacionados in df_juegos['Juegos relacionados']:
    if isinstance(lista_juegos_relacionados, list):
        for relacionados in lista_juegos_relacionados:
            if relacionados in conteo_juegos_relacionados:
                conteo_juegos_relacionados[relacionados] += 1
            else:
                conteo_juegos_relacionados[relacionados] = 1

df_juegos_relacionados = pd.DataFrame(list(conteo_juegos_relacionados.items()), columns=["Juegos relacionados", "Numero Juegos relacionados"])
df_juegos_relacionados = df_juegos_relacionados.sort_values(by="Numero Juegos relacionados", ascending=False).head(16)
df_juegos_relacionados

*Se muestra en un gráfico de barras los juegos relacionados más numerosos quitando los juegos de mesa base.*

In [ ]:
fig = px.bar(df_juegos_relacionados[1:],
             x                       = 'Juegos relacionados',
             y                       = 'Numero Juegos relacionados',  
             title                   = 'Top 15 juegos relacionados',
             opacity                 = 0.5,
             height                  = 700,
             color                   = 'Juegos relacionados'
            )

fig.update_xaxes(showticklabels=False)
fig.update_layout(xaxis = {'categoryorder' : 'total descending'}, title_x=0.5)

fig.show()

**Interpretación**

Los juegos relacionados más populares son "Juego de Tronos: LCG", "Canción de hielo y fuego: Caja de inicio", y "Arkham Horror: LCG" 
Hay un gran número de juegos relacionados con universos temáticos específicos, destacando los basados en Juego de Tronos, Arkham Horror y Star Wars. No es de extrañar que tanto Juego de Tronos como Star Wars aparezcan entre las palabras más populares para nombrar juegos.

Juegos tipo Living Card Game (LCG) tienen una presencia destacada, indicando un interés en juegos de cartas estratégicos y en la colección de cartas.

## ANÁLISIS DE LAS CATEGORÍAS
Géneros o temáticas asignadas a juegos de mesa. Hay un total de 81 categorías, pudiendo ser simultáneas.

*DataFrame para representar la distribución de juegos en diversas categorías.*

In [ ]:
conteo_categorias = {}

for lista_categorias in df_juegos["Categorias"]:
    if isinstance(lista_categorias, list):
        for categoria in lista_categorias:
            if categoria in conteo_categorias:
                conteo_categorias[categoria] += 1
            else:
                conteo_categorias[categoria] = 1

df_numero_categorias = pd.DataFrame(list(conteo_categorias.items()), columns=["Categorias", "Numero categorias"])
df_numero_categorias = df_numero_categorias.sort_values(by="Numero categorias", ascending=False)
df_numero_categorias #.head(15)

*A través de un gráfico de barras, se muestran las 15 categorías que aparecen más.*

In [ ]:
fig = px.bar(df_numero_categorias[:15],
             x                       = 'Categorias',
             y                       = 'Numero categorias',  
             title                   = 'Top 15 categorías en los distintos juegos',
             opacity                 = 0.5,
             height                  = 700,
             color_discrete_sequence = ['magenta']
            )
fig.update_layout(xaxis = {'categoryorder' : 'total descending'}, title_x=0.5)
fig.update_traces(showlegend=False)
fig.show()

**Interpretación**

Se observa una clara predominancia de las categorías relacionadas con cartas y fantasía. Las categorías de combate, ciencia ficción, wargame y miniaturas también son numerosas.

In [ ]:
categorias_lista= df_juegos["Categorias"].tolist()
categorias_unicas_conjuntos = set()

# Iterar sobre cada lista y convertirla en un conjunto, luego agregarlo al conjunto global
for lista in categorias_lista:
    categorias_unicas_conjuntos.add(tuple(set(lista)))

print(categorias_unicas_conjuntos)
print(len(categorias_unicas_conjuntos))

In [ ]:
frecuencia_categorias = {}

for lista in categorias_lista:
    conjunto = tuple(set(lista))
    if conjunto in frecuencia_categorias:
        frecuencia_categorias[conjunto] += 1
    else:
        frecuencia_categorias[conjunto] = 1

df_frecuencia_categorias = pd.DataFrame(list(frecuencia_categorias.items()), columns=['Categorias', 'Frecuencia'])
df_frecuencia_categorias = df_frecuencia_categorias.sort_values(by='Frecuencia', ascending=False)

df_frecuencia_categorias.head(20)

## ANÁLISIS DE LAS MECÁNICAS
Las normas que dirigen cómo los jugadores interactúan entre ellos y con el juego. Hay un total de 192 mecánicas, pudiendo ser simultáneas.


*Se crea un DataFrame del número de veces que aparece cada una y se muestran las 15 más populares.*

In [ ]:
conteo_mecanicas = {}

for lista_mecanicas in df_juegos["Mecanicas"]:
    if isinstance(lista_mecanicas, list):
        for mecanica in lista_mecanicas:
            if mecanica in conteo_mecanicas:
                conteo_mecanicas[mecanica] += 1
            else:
                conteo_mecanicas[mecanica] = 1

df_numero_mecanicas = pd.DataFrame(list(conteo_mecanicas.items()), columns=["Mecanicas", "Numero mecanicas"])
df_numero_mecanicas = df_numero_mecanicas.sort_values(by="Numero mecanicas", ascending=False).head(15)
df_numero_mecanicas

*Se filtran las mecánicas que aparecen en el DataFrame de arriba y se observa el número que hay en relación a la edición de juegos.*

In [ ]:
mecanicas_filtradas = df_numero_mecanicas["Mecanicas"].to_list()
df_mecanicas_filtradas = df_explode[df_explode["Mecanicas"].isin(mecanicas_filtradas)]
df_mecanicas_edicion = df_mecanicas_filtradas.groupby(["Mecanicas", "Edicion juegos"], as_index=False)['Link'].nunique().rename(columns={'Link': 'Count'})

df_mecanicas_edicion.head(8)

*La gráfica de barras muestra la cantidad de veces que se han registrado las mecánicas más populares en las diferentes ediciones de juegos.*

In [ ]:
fig = px.bar(df_mecanicas_edicion,
             x='Mecanicas',
             y='Count',  
             color='Edicion juegos',
             title='Número de mecánicas en relación a la edición de juegos',
             height=700
            )
fig.update_layout(xaxis = {'categoryorder' : 'total descending'}, title_x=0.5)
fig.show()

**Interpretación**

Destacan mecánicas como gestión de mano, lanzamiento de dados o habilidades variables; coincidiendo con la categoría predominante cartas.

No se observa una diferencia significativa entre cada mecánica y el número de edición de juegos; parece que los datos muestran un comportamiento similar.

*Se agruoan las mecánicas del juego, separándolas por edición de juegos.*

In [ ]:
df_juegos_relacion = df_explode.groupby(['Edicion juegos', 'Juegos relacionados'], as_index= False).nunique().sort_values('Link', ascending= False)
df_juegos_relacion.head()

*Se crea un gráfico de mapa de árbol interactivo para ver la relación que existe entre las reediciones y sus juegos base.*

In [ ]:
fig= px.treemap(data_frame = df_juegos_relacion[1:],
           values     = 'Link',
           path       = ['Edicion juegos','Juegos relacionados'],
           color      = 'Edicion juegos',
           hover_name= 'Link',
           custom_data= 'Link',
           title= "Análisis de la Relación entre Reediciones y Juegos Base"
           )    
fig.update_layout(title_x=0.5)

## ANÁLISIS DEL TIEMPO

*Se crea un gráfico de caja para observar el rango de duración.*

In [ ]:
fig= px.box(data_frame = df_juegos,
       x          = 'Mediana duracion',
       title= 'Duración de juegos')
fig.update_layout(title_x=0.5)

**Interpretación**

Se observa la presencia de un gran número de outliers en el tiempo de duración de los juegos, quedando el rango medio entre 1 minuto y 140; siendo la mediana de 45 minutos de juego.

Aunque existen valores extremos en el tiempo de duración de los juegos, la mayor concentración se encuentra en el rango medio, y la mediana proporciona una medida robusta del valor central del conjunto de datos.

Al buscar el juego con mayor duración, con valor atípico, se observa que no se trata de un error, ya que es un juego caracterizado por partidas de rol de una duración considerablemente extensa.

*Juego con mayor duración*

In [ ]:
df_juegos[(df_juegos['Mediana duracion'] == 5004.5)]

*Filtrado de los datos de la Mediana duración, eliminando los outliers.*

In [ ]:
Q1 = df_juegos['Mediana duracion'].quantile(0.25)
Q3 = df_juegos['Mediana duracion'].quantile(0.75)
IQR = Q3 - Q1

df_filtrados = df_juegos[(df_juegos['Mediana duracion'] >= Q1 - 1.5 * IQR) & (df_juegos['Mediana duracion'] <= Q3 + 1.5 * IQR)]

*Se crea un histrograma para representar las duraciones filtradas.*

In [ ]:
fig= px.histogram( data_frame= df_filtrados,
       x = 'Mediana duracion',
       opacity= 0.5,
       color_discrete_sequence= ['orange'],
       title= 'Duración de juegos')
fig.update_layout(title_x=0.5)

**Interpretación**

Se observa que la predominancia de las duraciones de los juegos está entre 30 y 60 minutos.

## CORRELACIONES ENTRE LOS DATOS

*Mapa de calor para examinar correlaciones lineales.*

In [ ]:
fig= px.imshow(img       = round(df_juegos[["Fecha publicacion", "Puntuacion", "Mediana jugadores", "Mediana duracion", "Edad", "Complejidad", "Dep idioma", 'Precio medio']].corr(), 1),
          text_auto = True, title= "Correlaciones Lineales")
fig.update_layout(title_x=0.5)

*La única correlación significativa mediante el coeficiente de Pearson es entre Complejidad y Edad, hay una correlación moderada positiva.*

In [ ]:
corr_complejidad_edad = df_juegos["Complejidad"].corr(df_juegos["Edad"])
corr_complejidad_edad

*Se crea un diagrama de dispersión. La línea representa la línea de regresión.*

In [ ]:
fig= px.scatter(data_frame  = df_juegos,
           x           = "Complejidad",
           y           = "Edad",
           title       = "Correlación lineal entre complejidad y edad",
           opacity     = 0.5,
           trendline   = 'ols')
fig.update_layout(title_x=0.5)

**Interpretación**

El gráfico representa que a medida que la complejidad de los juegos de mesa aumenta, la edad mínima recomendada de los jugadores también tiende a aumentar.

*Utilizando la correlación de Kendall se detecta una correlación no lineal entre complejidad y mediana duración.*

In [ ]:
lista_columnas = ['Edad', 'Puntuacion', 'Mediana duracion', 'Complejidad', 'Mediana jugadores', 'Dep idioma', 'Precio medio']

for i in lista_columnas:
    for j in lista_columnas:
        if i != j:  
            corr_columnas = df_juegos[i].corr(df_juegos[j], method='kendall')
            if corr_columnas >= 0.5:
                print(f"Correlación entre {i} y {j}: {corr_columnas}")

*Se utiliza matplotlib para representar con un gráfico de dispersión la correlación de Kendall.*

In [ ]:
sns.scatterplot(x='Complejidad', y='Mediana duracion', data=df_juegos)
plt.title('Correlación entre complejidad y mediana duración')
plt.xlabel('Complejidad')
plt.ylabel('Mediana duracion')

sns.scatterplot(x='Complejidad', y='Mediana duracion', data=df_juegos)

plt.show()

**Interpretación**

En la gráfica se observa que los juegos más complejos podrían tomar más tiempo en promedio que juegos menos complejos.

## ANÁLISIS DE LOS DATOS A LO LARGO DEL TIEMPO

*Fecha de creación de los juegos.*

In [ ]:
fecha_min= df_juegos["Fecha publicacion"].min()
fecha_max= df_juegos["Fecha publicacion"].max()
print(f"La fecha mínima de los juegos es: {fecha_min} y la máxima es: {fecha_max}.")

*Se crea un DataFrame con los juegos registrados en la página. Hay que tener en cuenta que 2024 ha iniciado ahora.*

In [ ]:
df_fecha_publicaciones = df_juegos.groupby('Fecha publicacion', as_index= False)['Link'].agg({'Link' : 'count'}).rename(columns= {'Link':'Publicaciones'})
df_fecha_publicaciones['Cumsum'] = df_fecha_publicaciones['Publicaciones'].cumsum()

df_fecha_publicaciones

*Representación las dos últimas décadas en una gráfica de rectas.*

In [ ]:
fig= px.line(data_frame = df_fecha_publicaciones[-25:],
        x          = 'Fecha publicacion',
        y          = 'Publicaciones',
        title   = 'Evolución de publicaciones en las dos úlimas décadas',
        height     =  700)
fig.update_layout(title_x=0.5)

**Interpretación**

Se observa el impacto de la pandemia de COVID-19, que inició a principios del año 2020 sobre el crecimiento exponencial de la creación de juegos ese año y la pronta recuperación del mercado.

*Gráfica de rectas para mostrar la evolución acumulativa de los juegos de mesa.*

In [ ]:
fig = px.line(
    data_frame=df_fecha_publicaciones,
    x='Fecha publicacion',
    y='Cumsum',
    title='Evolución acumulativa de publicaciones a lo largo de los años',
    height=700
)

fig.update_layout(title_x=0.5)
fig.show()

**Interpretación**

Se observa cómo a partir de 2020 empieza a haber un aumento, cada vez más pronunciado, en la creación y popularidad de juegos de mesa. Puede que el tiempo en casa durante la pandemia haya favorecido a ello.

*DataFrame que muestra el número de ediciones de juegos en diferentes años.*

In [ ]:
df_fechas_edicion = df_explode.groupby(['Edicion juegos', 'Fecha publicacion'], as_index=False)['Link'].nunique().rename(columns={'Link': 'Count'})
df_fechas_edicion['Cumulative Count'] = df_fechas_edicion.groupby('Edicion juegos')['Count'].cumsum()

df_fechas_edicion

*Gráfica de líneas que ilustra la relación entre la fecha de publicación y el número de ediciones de juegos en las últimas décadas, con el propósito de observar el posible impacto del COVID-19 en la edición de juegos.*

In [ ]:
df_fecha_filtrada = df_fechas_edicion[df_fechas_edicion['Fecha publicacion'] >= 2010]

fig = px.line(data_frame=df_fecha_filtrada, x='Fecha publicacion', y='Count', color='Edicion juegos', height=700, title='Impacto de la Pandemia en la Edición de Juegos de Mesa',
)
fig.update_layout(title_x=0.5)

fig.add_vline(x=2020, line_dash="dash", line_color="red", name="Covid")

fig.show()

**Interpretación**

Se aprecia un evidente crecimiento en la industria de los juegos de mesa, experimentando un éxito constante y en ascenso, con la excepción de la pandemia. Durante este período, las ediciones de juegos más afectadas fueron los juegos de mesa base; sin embargo, también son los que muestran una mayor recuperación posteriormente.

La disminución en el año 2023 puede atribuirse a la posibilidad de que no estén registrados en la página todos los juegos recién creados, mientras que la reducción en 2024 puede deberse al comienzo del año.

*Suma acumulativa de la edición de juegos.*

In [ ]:
fig= px.line(data_frame = df_fechas_edicion,
        x          = 'Fecha publicacion',
        y          = 'Cumulative Count',
        color      = 'Edicion juegos',
        title      = "Suma Acumulativa en la Edición de Juegos",
        height     =  700)

fig.update_layout(title_x=0.5)


**Interpretación**

Es evidente que la creación de juegos ha tenido un crecimiento rápido, sobre todo en la última década.

*Gráfica interactiva*
- *Se observa una tendencia de aumento similar en el uso de las mecánicas de juego, especialmente en las más populares.*

In [ ]:
df_mecanicas = df_explode.groupby(['Mecanicas', 'Fecha publicacion'], as_index = False)['Link'].nunique().rename(columns={'Link': 'Count'})

fig= px.line(data_frame = df_mecanicas,
        x          = 'Fecha publicacion',
        y          = 'Count',
        color      = 'Mecanicas',
        height     =  700,
        title= "Análisis Interactivo de Tendencias en el Uso de Mecánicas de Juego")
fig.update_layout(title_x=0.5)


*Gráfica interactiva*
- *Se evidencia una tendencia de aumento similar en el uso de las categorías, especialmente en las más populares.*

In [ ]:

df_categorias = df_explode.groupby(['Categorias', 'Fecha publicacion'], as_index = False)['Link'].nunique().rename(columns={'Link': 'Count'})

fig= px.line(data_frame = df_categorias,
        x          = 'Fecha publicacion',
        y          = 'Count',
        color      = 'Categorias',
        height     =  1000,
        title= "Exploración Interactiva de Tendencias en el Uso de Categorías")
fig.update_layout(title_x=0.5)

## ANÁLISIS DEL PRECIO

*Se muestran los nombres de los juegos y los precios. Los precios más bajos representan expansiones de cartas y los más caros suelen atribuirse a colecciones grandes de miniaturas y mapas.*

In [ ]:
df_precios = df_juegos.sort_values(by="Precio medio")
columnas_nombre_precio = ["Nombre", "Precio medio"]

df_precios[columnas_nombre_precio]

*Histograma para analizar los precios de los juegos, se representan mediana (verde) y media (rojo).*

In [ ]:
fig = px.histogram(data_frame=df_precio, x='Precio medio')

media = np.mean(df_precio['Precio medio'])
mediana = np.median(df_precio['Precio medio'])

fig.add_vline(x=media, line_dash="dash", line_color="red", name="Media")
fig.add_vline(x=mediana, line_dash="dash", line_color="green", name="Mediana")

fig.update_layout(title='Histograma de Precios Medios', title_x=0.5)
fig.update_traces(opacity=0.5)
fig.show()

**Interpretación**

Se observa una clara tendencia en los precios en el rango de 10 a 30 euros.

Es un histograma desplazado a la derecha. Se observa una diferencia entre media y mediana, la media se encuentra desplazada hacia la derecha.
 

*Diagrama de cajas para observar más a fondo los precios.*

In [ ]:
fig = px.box(data_frame= df_precio,
                   x = 'Precio medio')

fig.update_layout(title='Diagrama de Caja de Precios Medios', title_x=0.5)
fig.show()

**Interpretación**

La mediana de los precios es de 26 euros. A partir de 80 euros se consideran los precios de los juegos como outliers.

## ANÁLISIS DE LAS PUNTUACIONES

*Puntuaciones más altas de los juegos separados por ediciones de juego.*

In [ ]:
df_mas_puntuacion = df_juegos.sort_values(by='Puntuacion', ascending=False)
df_mas_puntuacion[["Puntuacion", "Nombre", "Autores", "Edicion juegos"]].head(10)

**Interpretación**

La lista destaca juegos de mesa bien valorados, como "Terraforming Mars: Preludio" y "Pandemic Legacy: Temporada 1". 

La presencia destacada de expansiones y reimplementaciones entre las mejores puntuaciones indica que probablemente los jugadores adquieren estas opciones con la certeza de que disfrutan del juego base.

*Histograma para ver las puntuaciones de los juegos separados por ediciones de juego.*

In [ ]:
fig = px.histogram(data_frame=df_juegos, x='Puntuacion', facet_col='Edicion juegos',color = 'Edicion juegos')

fig.update_layout(
    title='Puntuaciones en las distintas ediciones de juegos',
    xaxis_title='Puntuacion',
    yaxis_title='Frecuencia',
    bargap=0.1,
    bargroupgap=0.1,
    title_x=0.5
)

fig.update_traces(opacity=0.5, showlegend=False)
fig.show()

**Interpretación**

Las Expansiones de juegos son un mercado menos homogeneo que los juegos base, pero suele tener mayor éxito, esto puede deberse a la aceptación previa del juego base. Se observa que no hay datos significativos con Reimplementación  Integración de juegos, sin embargo encabezan las listas junto con Expansión.

## CONCLUSIONES DEL ANÁLISIS

**Tendencias en la Industria:**

Hay una tendencia creciente en la producción de juegos de mesa, sugiriendo un aumento en el interés en la industria en los últimos años. La mayoría de los juegos están diseñados para grupos más pequeños (2-6 personas), con una experiencia de juego de duración moderada, lo que los hace adaptables a un tiempo limitado disponible.

**Accesibilidad y Diversidad:**

Los juegos son adecuados tanto para niños como para adultos, presentan baja complejidad y dependencia del idioma, haciéndolos accesibles para un público amplio. La diversidad en las categorías de ediciones de juegos revela distintas estrategias y enfoques en la industria.

**Autores y Contribuciones:**

Un número significativo de juegos carece de autor identificado. Eric M. Lang destaca como el autor con el mayor número de juegos, abarcando diversas temáticas y universos.

**Temáticas y Franquicias:**

Tendencias notables incluyen un interés en juegos de cartas coleccionables (LCG), franquicias populares como Star Wars y Marvel, y temáticas de fantasía, especialmente relacionadas con Juego de Tronos.
Destacan mecánicas como la gestión de mano, lanzamiento de dados y habilidades variables.

**Crecimiento y Pandemia:**

Se observa un crecimiento constante en la industria de los juegos de mesa, con un aumento significativo a partir de 2020, posiblemente atribuido al tiempo en casa durante la pandemia de COVID-19. La pandemia afectó sobre todo a las ediciones de juegos en 2020, pero hubo una pronta recuperación posteriormente.

**Precios y Valoración:**

Hay una clara tendencia en los precios en el rango de 10 a 30 euros, con juegos bien valorados como "Terraforming Mars: Preludio" y "Pandemic Legacy: Temporada 1".
La presencia destacada de expansiones y reimplementaciones entre las mejores puntuaciones indica que los jugadores adquieren estas opciones con la certeza de que disfrutan del juego base.

**Impacto de la Complejidad y Duración:**

La mayoría de los juegos tienen una duración concentrada entre 30 y 60 minutos, con una mediana de 45 minutos. A medida que la complejidad de los juegos de mesa aumenta, la edad de los jugadores también tiende a aumentar. Los juegos más complejos podrían tomar más tiempo en promedio.

En resumen, la industria de juegos de mesa muestra un crecimiento constante, adaptándose a una variedad de públicos con juegos accesibles y diversificados. Las tendencias indican un interés en temáticas específicas, franquicias populares y un enfoque en juegos de duración moderada. La pandemia parece haber influido positivamente en el aumento de la producción y popularidad de juegos de mesa.